# Monitoring an ongoing MIBI run

This notebook can be run alongside an active MIBIScope run.  As images are generated, this notebook will automatically pass the data through pre-specified functions, like tiff 
extraction, qc metric computation, etc. Eventually, all of the processing steps in the toffy repo will be incorporated into the Watcher functionality here. For now, this notebook will automatically extract tiffs, compute pulse height data, stitch images by channel, and generate the QC and MPH plots of your data. 

In [1]:
import os

from toffy.panel_utils import load_panel
from toffy.fov_watcher import start_watcher
from toffy.watcher_callbacks import build_callbacks

## Required variables

 - `run_name` should contain the exact name of the MIBI run that will be monitored
 - `panel_path` should point to a panel csv specifying the targets on your panel. You can download your panel online from the Ionpath MibiTracker and then copy the file to the `C:\\Users\\Customer.ION\\Documents\panel_files` directory (see [panel format](https://github.com/angelolab/toffy#panel-format) for more information)
  - `extract_prof` indicates whether you want to include mass proficient extraction on top of the default (mass deficient) extraction

In [ ]:
# the name of the run
run_name = 'YYYY-MM-DD_run_name'

# path to user panel
panel_path = 'C:\\Users\\Customer.ION\\Documents\\panel_files\\my_cool_panel.csv'

# whether to include proficient extraction or not
extract_prof = True

Everything necessary for and subsequently outputted from this notebook is stored in the automatic directories established in `1_set_up_toffy.ipynb`. More information on the uses and locations of the directories in toffy can be found in the [README](https://github.com/angelolab/toffy#directory-structure).

In [ ]:
# load panel
panel = load_panel(panel_path)

# these are set automatically
base_dir = os.path.join('D:\\Data', run_name)
extraction_dir = os.path.join('D:\\Extracted_Images', run_name)
metrics_data_dir = os.path.join('C:\\Users\\Customer.ION\\Documents\\run_metrics', run_name, 'fov_data')
metrics_plot_dir = os.path.join('C:\\Users\\Customer.ION\\Documents\\run_metrics', run_name)

# path to log folder
log_path = os.path.join('C:\\Users\\Customer.ION\\Documents\\watcher_logs', run_name)

In [ ]:
# define the specific channels you want to extract as intensity images, leave list empty if there are none
# NOTE: these must match names found in panel_path
intensity_channels = ["Au", "chan_39"]

## Callback initialization

Callbacks listed in `fov_callbacks` arugment will be run on each completed FOV, the moment both the .bin & .json are found.

* The `extract_tiffs` callback specifies that every FOV generates tiffs according to the supplied `panel`. <br/> (See [3b_extract_images_from_bin](./3b_extract_images_from_bin.ipynb) for more details.)

* The `generate_pulse_heights` FOV callback computes the median pulse heights for each mass specified in the `panel`. 
<br/> (See [4b_normalize_image_data](./4b_normalize_image_data.ipynb) for more details.)

Callbacks listed in the `intermediate_callbacks` argument run and update as each new FOV is generated.

* The `plot_qc_metrics` intermediate callback will run all currently available qc metrics on each FOV and plot the results. <br/> (See [3c_generate_qc_metrics](./3c_generate_qc_metrics.ipynb) for more details.)

* The `plot_mph_metrics` intermediate callback will compute the median pulse height data for each  FOV and plot the results. Additional arguments are: `regression` which when set to True will also plot the linear regression line for the data. <br/> (See [3d_compute_median_pulse_height](./3d_compute_median_pulse_height.ipynb) for more details.)

Callbacks listed in the `run_callbacks` argument will be run only once all expected FOV's have been discovered and processed. 

* The `image_stitching` run callback will create a single image, which stitched together all FOV images for a specific channel. Additional arguments are: `channels`. <br/> (See [3e_stitch_images](./3e_stitch_images.ipynb) for more details.)

* The `check_incomplete_fovs` run callback will check the run for any partially generated images. <br/> (See [3b_extract_images_from_bin](./3b_extract_images_from_bin.ipynb) for more details.)
  
* The `check_missing_fovs` run callback checks that the run produces the appropriate .bin and .json all files for all FOVs included in the run file. <br/> (See [3f_FOV_checks](./3f_FOV_checks.ipynb) for more details.)

In [ ]:
fov_callback, run_callback, intermediate_callback = build_callbacks(
    run_callbacks = ['image_stitching', 'check_incomplete_fovs', 'check_missing_fovs'],
    intermediate_callbacks = ['plot_qc_metrics', 'plot_mph_metrics'],
    fov_callbacks = ['extract_tiffs', 'generate_pulse_heights'],
    extract_prof=extract_prof,
    tiff_out_dir=extraction_dir,
    qc_out_dir=metrics_data_dir,
    mph_out_dir=metrics_data_dir,
    pulse_out_dir=metrics_data_dir,
    plot_dir=metrics_plot_dir,
    save_dir=metrics_plot_dir,
    panel=panel,
    warn_overwrite=False,
    intensities=intensity_channels,
)

In [ ]:
start_watcher(base_dir, log_path, fov_callback, run_callback, intermediate_callback)